# Generate SVJ data (MG5 -> Pythia -> Delphes)

## Generate LHE from Madgraph

### Pre-requisites

We can use Madgraph to generate parton-level (LHE) files. Then shower/reco will be performed separately with Pythia and Delphes. Install a copy of Madgraph and instal Pythia/Delphes

### Define import directories for process files, madgraph, etc

In [19]:
import os
# Ideally, you should only need to explicitly define your svj_path and the rest 
# of the paths should be made for you from that
svj_path = "/home/tfaucett/Projects/svj-production"
gen_path = svj_path+"/gen"
mg5_path = gen_path+"/HEPTools/MG5"
pythia_path = gen_path+"/HEPTools/MG5/HEPTools/pythia8"
log_path = gen_path+"/logs"
set_pythia = "export PYTHIA8="+pythia_path
DelphesPythia8 = mg5_path+"/Delphes/DelphesPythia8"

# Locations for custom run files and process information
proc_path = gen_path+"/process_files"
cmnd_path = proc_path+"/cmnd_files"
lhe_path = proc_path+"/lhe_proc"

# Set Pythia path for use with Delphes
os.system(set_pythia)

0

## Generate LHE file. 

Jupyter/python isn't really the best way to interact with Madgraph. So the code below will execute the terminal commands to run madgraph, but any changes need to be made via the appropriate run cards/process files/etc. If you decide to run this through Jupyter, you can monitor progress via log files in the directory "log/mg5".

### Custom madgraph settings

In [20]:
from dict_replace import dict_replace

# Load the MG5 input template
mg5_template = proc_path+"/lhe_proc/templates/schannel_template.txt"
d = {
    "nevents":20000,
    "ebeam1":6500.0,
    "ebeam2":6500.0,
    "xqcut":100,
    "ptj":50,
    "MY1":1500, # Z' mass
    "MXd":10 # dark quark mass
}

process_file = "%s/lhe_proc/lhe_Zp_%s.txt" %(proc_path, d["MY1"])

dict_replace(mg5_template, process_file, d)
%less $process_file

set default_unset_couplings 99
set group_subprocesses Auto
set ignore_six_quark_processes False
set loop_optimized_output True
set low_mem_multicore_nlo_generation False
set loop_color_flows False
set gauge unitary
set complex_mass_scheme False
set max_npoint_for_channel 0

import model DMsimp_s_spin1
define p = g u c d s u~ c~ d~ s~
define j = g u c d s u~ c~ d~ s~
define l+ = e+ mu+
define l- = e- mu-
define vl = ve vm vt
define vl~ = ve~ vm~ vt~
define p = 21 2 4 1 3 -2 -4 -1 -3 5 -5 # pass to 5 flavors
define j = p
define j = g u c d b s t u~ c~ d~ b~ s~ t~
generate p p > xd xd~ @0
add process p p > xd xd~ j @1
add process p p > xd xd~ j j @2

output /home/tfaucett/Projects/svj-production/data/raw_data/sig_schannel
launch /home/tfaucett/Projects/svj-production/data/raw_data/sig_schannel

set nevents 20000
set run_card ebeam1 6500.0
set run_card ebeam2 6500.0
set pdlabel nn23lo1 
set ickkw 1 
set xqcut 100
set ptj 50
set asrwgtflavor 5
set maxjetflavor 5

set MY1 1500
set MXd 10
set

## Run MG5 and generate LHE file

In [14]:
print("Using proc_file " + process_file)
log_file = log_path+"/mg5/log.txt"
run_cmnd = "nohup "+ mg5_path +"/bin/mg5_aMC " + process_file + " > " + log_file
print("You can follow the progress with the command:")
print("tail -f "+ log_path + "/mg5/log.txt")
os.system(run_cmnd)

Using proc_file /home/tfaucett/Projects/svj-production/gen/process_files/lhe_proc/lhe_Zp_1500.txt
You can follow the progress with the command:
tail -f /home/tfaucett/Projects/svj-production/gen/logs/mg5/log.txt


0

## Swap PDG ID
Once the LHE file is generated, we have to unzip the LHE file and swap PDG IDs for the upcoming pythia process to make the correct modifications for the SVJ/HiddenValley process

*Note, if you are using a Mac you will want to install gsed instead of using the built-in sed. This can be done with homebrew 'brew install gsed'*

In [15]:
lhe_path = svj_path + "/data/raw_data/sig_schannel/Events/run_01"
lhe_file = lhe_path + "/unweighted_events.lhe"
if not os.path.isfile(lhe_file):
    print("unzipping "+lhe_file+".gz")
    os.system("gunzip "+lhe_file+".gz")
print("replacing PDG ID in " + lhe_file)
os.system("sed -i 's/5000521/4900101/g' " + lhe_file)

unzipping /home/tfaucett/Projects/svj-production/data/raw_data/sig_schannel/Events/run_01/unweighted_events.lhe.gz
replacing PDG ID in /home/tfaucett/Projects/svj-production/data/raw_data/sig_schannel/Events/run_01/unweighted_events.lhe


0

## Run the LHE file with Pythia8 and Delphes

It's easiest to run pythia through Delphes. Assuming you already installed Delphes and Pythia8 through Madgraph previously, the two can be linked by simply re-running "make" with the correct Pythia8 tag: https://cp3.irmp.ucl.ac.be/projects/delphes/wiki/WorkBook/Pythia8

*./configure --prefix=path_to_PYTHIA8_installation*

*export PYTHIA8=/home/tfaucett/Projects/svj-production/gen/HEPTools/MG5/HEPTools/pythia8*

*cd MG5/Delphes*

*make HAS_PYTHIA8=true*

### custom pythia cmnd file

The SVJ/Hidden Valley model in Pythia needs r_inv, Z' mass, etc explicitly defined and referenced in a cmnd file. The code below can be generate this cmnd file by taking a template file and inserting the desired values.

In [21]:
import shutil
import fileinput

def gen_dict(nEvents, mY1, mXd, rinv, lambdas, out_path):
    # Defines the dictionary with values to 
    # replace in the template file
    d = {
        "nEvents":nEvents,
        "lambdas":lambdas,
        "alpha_fsr":0.13,
        "pTminFSR":lambdas*1.1,
        "Z_prime":mY1,
        "dark_quark":mXd,
        "dark_meson":2.0*mXd,
        "mWidth":mXd/50.0,
        "mMin":mXd - mXd / 50.0,
        "mMax":mXd + mXd / 50.0,
        "dark_matter":mXd - 0.01,
        "1_rinv": 1.0 - rinv,
        "rinv": rinv,
        "1_rin_half":round((1 - rinv) / 5.,6),
        "output_path":out_path
    }
    return d

def cmnd_generator(nEvents, mY1, mXd, rinv, lambdas, out_path):
    # Grab the template cmnd file path
    cmnd_template = cmnd_path + "/templates/rinv_template.cmnd"
    
    # Define a new output cmnd file (with chosen paramters in the file name)
    new_cmnd = "%s/SVJ_n_%s_mZ_%s_mXd_%s_rinv_%s_lam_%s.cmnd" %(cmnd_path, nEvents, mY1, mXd, rinv, lambdas)
    
    # Build the dictionary of replacement values
    dictionary = gen_dict(nEvents, mY1, mXd, rinv, lambdas, out_path)
    
    # Read in the template file
    with open(cmnd_template) as f:
        data = f.read()
        
    # Make substitutions in the template file text
    for k,v in dictionary.items():
        data = data.replace("$"+str(k), str(v))
    
    # Save the modified template as a new file
    with open(new_cmnd, 'w') as writer:
        writer.write(data)
        
    return new_cmnd

### Run Pythia/Delphes on LHE
now we can run the Delphes command on the LHE file with Pythia included, referencing the appropriate pythia8 card and Delphes tcl file

In [23]:
def generate_events(cmnd_file, nEvents, mY1, mXd, rinv, lambdas):
    delphes_outfile = os.path.basename(cmnd_file).split("SVJ_")[-1].split(".cmnd")[0]
    delphes_outfile = svj_path + "/data/root_files/delphes_" + delphes_outfile
    delphes_outfile = delphes_outfile.replace(".", "p")
    delphes_outfile_ext = delphes_outfile + ".root"
#     if os.path.isfile(delphes_outfile_ext):
#         os.remove(delphes_outfile_ext)
    if not os.path.isfile(delphes_outfile_ext):
        gen_log_file = log_path + "/delphes/%s.log" % (delphes_outfile.split("/")[-1])
        run_cmnd = "nohup %s %s %s %s > %s 2>&1 &" %(DelphesPythia8, delphes_card, cmnd_file, delphes_outfile_ext, gen_log_file)
        os.system(run_cmnd)
        print("You can check progress with the command")
        tail_command = "tail -f " + gen_log_file
        print(tail_command)



# delphes_card = proc_path + "/delphes_cards/delphes_card_ATLAS_jpt200.tcl"
delphes_card = f"{proc_path}/delphes_cards/dark-showers/delphes_card_CMS.tcl"
# delphes_card = proc_path + "/delphes_cards/default_cards/delphes_card_ATLAS.tcl"
nEvents = 20000
mY1 = 1500
mXds = [5, 10]
rinvs = [0.0, 0.3, 0.6, 1.0]
lambda_list = [5, 10]
for rinv in rinvs:
    for lambdas in lambda_list:
        for mXd in mXds:
            out_path = svj_path + "/data/raw_data/sig_schannel/Events/run_01/unweighted_events.lhe"
            cmnd_file = cmnd_generator(nEvents, mY1, mXd, rinv, lambdas, out_path)
            # generate_events(cmnd_file, nEvents, mY1, mXd, rinv, lambdas)
%less $cmnd_file

! 1) Settings used in the main program.

Main:numberOfEvents = 20000         ! number of events to generate
Main:timesAllowErrors = 500          ! how many aborts before run stops

! 2) Settings related to output in init(), next() and stat().

Init:showChangedSettings = on      ! list changed settings
Init:showChangedParticleData = off ! list changed particle data
Next:numberCount = 1000             ! print message every n events
Next:numberShowInfo = 1            ! print event information n times
Next:numberShowProcess = 1         ! print process record n times
Next:numberShowEvent = 1           ! print event record n times
PDF:pSet = 13 




! Matching
!JetMatching:doVeto=off
JetMatching:qCut         = 100
JetMatching:nJetMax      = 2
JetMatching:setMad=on
JetMatching:merge=on
JetMatching:scheme=1
JetMatching:jetAlgorithm=2




! Hidden Valley
HiddenValley:Ngauge =2
HiddenValley:nFlav =1
HiddenValley:alphaOrder =1
HiddenValley:Lambda = 10
HiddenValley:alphaFSR =0.13
HiddenValley:spin